In [13]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

df = pd.read_csv('../data/cleaned_global_17-24.csv')


df.head()

,artist_names,track_name,peak_streams,total_streams,max_peak_rank,min_peak_rank,max_days_on_chart,best_day_streams,first_appearance,uri,source,count,best_day_date
0,The Weeknd,Blinding Lights,8453567,4552239347,8,1,1858,8453567,2019-11-29,spotify:track:0sf12qNH5qcw8qpgymFOqD,Republic Records,1858,2020-03-20
1,Harry Styles,As It Was,16103849,3640335748,40,1,1002,16103849,2022-03-31,spotify:track:4LRPiXqCikLlN15c3yImP7,Columbia,1002,2022-04-01
2,Lewis Capaldi,Someone You Loved,3974554,3625415177,198,4,2166,3974554,2019-01-07,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,Vertigo Berlin,2166,2019-10-30
3,Ed Sheeran,Shape of You,9891056,3607629552,1,1,2407,9891056,2017-01-06,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Atlantic Records UK,2407,2017-03-03
4,"Post Malone, Swae Lee",Sunflower - Spider-Man: Into the Spider-Verse,5033261,3467020076,196,1,2187,5033261,2018-10-18,spotify:track:1A6OTy97kk0mMdm78rHsm8,Republic Records,2187,2019-01-18


In [14]:

import requests
import base64

auth_string = f"{client_id}:{client_secret}"
auth_bytes = auth_string.encode("utf-8")
auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {auth_base64}",
    "Content-Type": "application/x-www-form-urlencoded"
}
data = {"grant_type": "client_credentials"}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    token_info = response.json()
    access_token = token_info["access_token"]
    print(f"Access Token: {access_token}")
else:
    print("Failed to get token:", response.status_code, response.json())


Access Token: BQB72l6R8MSgBj6IdHek6Wi1QJltyOI1M8SuDxmS2mNi8OGlPImGKF7rRRFWf0Ml2HLH9nMhOmLwbhfRALWsoZK4Dh51IapCw6fc6dLDEbe7v8fqYj4UyPJQwxWG7xZxkttl7csfliI


In [17]:
head = df.head(150).copy()
head['uri'].head()

0    spotify:track:0sf12qNH5qcw8qpgymFOqD
1    spotify:track:4LRPiXqCikLlN15c3yImP7
2    spotify:track:2TIlqbIneP0ZY1O0EzYLlc
3    spotify:track:7qiZfU4dY1lWllzX7mPBI3
4    spotify:track:1A6OTy97kk0mMdm78rHsm8
Name: uri, dtype: object

In [18]:
head['uri'].head().apply(lambda x: x.split(':')[-1])

0    0sf12qNH5qcw8qpgymFOqD
1    4LRPiXqCikLlN15c3yImP7
2    2TIlqbIneP0ZY1O0EzYLlc
3    7qiZfU4dY1lWllzX7mPBI3
4    1A6OTy97kk0mMdm78rHsm8
Name: uri, dtype: object

In [19]:
track_ids = df['uri'].apply(lambda x: x.split(':')[-1]).tolist()

In [7]:
track_ids

['0sf12qNH5qcw8qpgymFOqD',
 '4LRPiXqCikLlN15c3yImP7',
 '2TIlqbIneP0ZY1O0EzYLlc',
 '7qiZfU4dY1lWllzX7mPBI3',
 '1A6OTy97kk0mMdm78rHsm8',
 '5HCyWlXZPP0y6Gqq8TgA20',
 '0CcQNd8CINkwQfe1RDtGV6',
 '0u2P5u6lvoDfwTYjAADbn4',
 '0tgVpDi06FyKpA1z0VMD4v',
 '02MWAaffLxlfxAUY7c5dvx',
 '2QjOHCTQ1Jl3zawyYOpxh6',
 '1e9oZCCiX42nJl0AcqriVo',
 '1rgnBhdG2JDFTbYkYRZAku',
 '5aAx2yezTd8zXrkmtKl66Z',
 '7jtQIBanIiJOMS6RyCx6jZ',
 '0yLdNVWF3Srea0uzk55zFn',
 '5uCax9HTNlzGybIStD3vDh',
 '1Qrg8KqiBpW07V7PNxwwwL',
 '1BxfuPKGuaTgP7aM0Bbdwr',
 '7a8GawTVgHZvBLQOh46he7',
 '6WrI0LAC5M1Rw2MnX2ZvEg',
 '7wGoVu4Dady5GV0Sv4UIsx',
 '0TK2YIli7K1leLovkQiNik',
 '3AJwUDP919kvQ9QcozQPxg',
 '6zSpb8dQRaw0M1dK8PBwQz',
 '6RUKPb4LETWmmr3iAEQktW',
 '2LBqCSwhJGcFQeTHMVGwy3',
 '5ww2BF9slyYgNOk37BlC4u',
 '4VginDwYTP2eaHJzO0QMjG',
 '2Fxmhks0bxGSBdJ92vM42m',
 '32iYwQ4OYiurnGnOfJ1aEh',
 '4uUG5RXrOk84mYEfFvj3cK',
 '6gBFPUFcJLzWGx4lenP6h2',
 '5QO79kh1waicV47BqGRL3g',
 '2qSkIjg1o9h3YT9RAgYN75',
 '6dOtVTDdiauQNBQEDOtlAB',
 '7yq4Qj7cqayVTp3FF9CWbm',
 

In [20]:
import time

chunk_size = 50
chunks = [track_ids[i:i + chunk_size] for i in range(0, len(track_ids), chunk_size)]

all_track_data = []

headers = {
    "Authorization": f"Bearer {access_token}"
}

def handle_rate_limit(response):
    if response.status_code == 429:
        retry_after = int(response.headers.get("Retry-After", 5))
        print(f"Rate limit hit! Waiting {retry_after} seconds...")
        time.sleep(retry_after)

for i, chunk in enumerate(chunks):
    url = f"https://api.spotify.com/v1/tracks?ids={','.join(chunk)}"
    print(f"Fetching chunk {i+1}/{len(chunks)}...")

    response = requests.get(url, headers=headers)

    handle_rate_limit(response)

    if response.status_code == 200:
        track_data = response.json()
        all_track_data.extend(track_data['tracks'])
        print(f" Fetched {len(track_data['tracks'])} tracks.")
    else:
        print(f" Error: {response.status_code}, {response.text}")
        break

print(f" Total tracks fetched: {len(all_track_data)}")


Fetching chunk 1/183...
 Fetched 50 tracks.
Fetching chunk 2/183...
 Fetched 50 tracks.
Fetching chunk 3/183...
 Fetched 50 tracks.
Fetching chunk 4/183...
 Fetched 50 tracks.
Fetching chunk 5/183...
 Fetched 50 tracks.
Fetching chunk 6/183...
 Fetched 50 tracks.
Fetching chunk 7/183...
 Fetched 50 tracks.
Fetching chunk 8/183...
 Fetched 50 tracks.
Fetching chunk 9/183...
 Fetched 50 tracks.
Fetching chunk 10/183...
 Fetched 50 tracks.
Fetching chunk 11/183...
 Fetched 50 tracks.
Fetching chunk 12/183...
 Fetched 50 tracks.
Fetching chunk 13/183...
 Fetched 50 tracks.
Fetching chunk 14/183...
 Fetched 50 tracks.
Fetching chunk 15/183...
 Fetched 50 tracks.
Fetching chunk 16/183...
 Fetched 50 tracks.
Fetching chunk 17/183...
 Fetched 50 tracks.
Fetching chunk 18/183...
 Fetched 50 tracks.
Fetching chunk 19/183...
 Fetched 50 tracks.
Fetching chunk 20/183...
 Fetched 50 tracks.
Fetching chunk 21/183...
 Fetched 50 tracks.
Fetching chunk 22/183...
 Fetched 50 tracks.
Fetching chunk 23/1

In [ ]:
import json

first_object = all_track_data[0]
print(json.dumps(first_object, indent=4))


{
    "album": {
        "album_type": "album",
        "artists": [
            {
                "external_urls": {
                    "spotify": "https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ"
                },
                "href": "https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ",
                "id": "1Xyo4u8uXC1ZmMpatF05PJ",
                "name": "The Weeknd",
                "type": "artist",
                "uri": "spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ"
            }
        ],
        "available_markets": [
            "AR",
            "AU",
            "AT",
            "BE",
            "BO",
            "BR",
            "BG",
            "CA",
            "CL",
            "CO",
            "CR",
            "CY",
            "CZ",
            "DK",
            "DO",
            "DE",
            "EC",
            "EE",
            "SV",
            "FI",
            "FR",
            "GR",
            "GT",
            "HN",
            "HK",
 

In [21]:
track_data = pd.json_normalize(all_track_data)

In [22]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
track_data.head()

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album.album_type,album.artists,album.available_markets,album.external_urls.spotify,album.href,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,[KR],1,201573,False,https://api.spotify.com/v1/tracks/0sf12qNH5qcw...,0sf12qNH5qcw8qpgymFOqD,False,Blinding Lights,22,None,1,track,spotify:track:0sf12qNH5qcw8qpgymFOqD,single,[{'external_urls': {'spotify': 'https://open.s...,[KR],https://open.spotify.com/album/2ZfHkwHuoAZrlz7...,https://api.spotify.com/v1/albums/2ZfHkwHuoAZr...,2ZfHkwHuoAZrlz7RMj0PDz,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Blinding Lights,2019-11-29,day,1,album,spotify:album:2ZfHkwHuoAZrlz7RMj0PDz,USUG11904206,https://open.spotify.com/track/0sf12qNH5qcw8qp...
1,[{'external_urls': {'spotify': 'https://open.s...,[],1,167303,False,https://api.spotify.com/v1/tracks/4LRPiXqCikLl...,4LRPiXqCikLlN15c3yImP7,False,As It Was,16,None,1,track,spotify:track:4LRPiXqCikLlN15c3yImP7,single,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/2pqdSWeJVsXAhHF...,https://api.spotify.com/v1/albums/2pqdSWeJVsXA...,2pqdSWeJVsXAhHFuVLzuA8,[{'url': 'https://i.scdn.co/image/ab67616d0000...,As It Was,2022-03-31,day,1,album,spotify:album:2pqdSWeJVsXAhHFuVLzuA8,USSM12200612,https://open.spotify.com/track/4LRPiXqCikLlN15...
2,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,182160,False,https://api.spotify.com/v1/tracks/2TIlqbIneP0Z...,2TIlqbIneP0ZY1O0EzYLlc,False,Someone You Loved,70,None,3,track,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,single,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/0NVQ9k3wKmuK6T0...,https://api.spotify.com/v1/albums/0NVQ9k3wKmuK...,0NVQ9k3wKmuK6T02lLMl6y,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Breach,2018-11-08,day,4,album,spotify:album:0NVQ9k3wKmuK6T02lLMl6y,DEUM71807062,https://open.spotify.com/track/2TIlqbIneP0ZY1O...
3,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,233712,False,https://api.spotify.com/v1/tracks/7qiZfU4dY1lW...,7qiZfU4dY1lWllzX7mPBI3,False,Shape of You,87,None,4,track,spotify:track:7qiZfU4dY1lWllzX7mPBI3,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/3T4tUhGYeRNVUGe...,https://api.spotify.com/v1/albums/3T4tUhGYeRNV...,3T4tUhGYeRNVUGevb0wThu,[{'url': 'https://i.scdn.co/image/ab67616d0000...,÷ (Deluxe),2017-03-03,day,16,album,spotify:album:3T4tUhGYeRNVUGevb0wThu,GBAHS1600463,https://open.spotify.com/track/7qiZfU4dY1lWllz...
4,[{'external_urls': {'spotify': 'https://open.s...,[],1,158053,False,https://api.spotify.com/v1/tracks/1A6OTy97kk0m...,1A6OTy97kk0mMdm78rHsm8,False,Sunflower - Spider-Man: Into the Spider-Verse,3,None,1,track,spotify:track:1A6OTy97kk0mMdm78rHsm8,single,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/47LpgGVshd0tbFS...,https://api.spotify.com/v1/albums/47LpgGVshd0t...,47LpgGVshd0tbFSbm9tTLb,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Sunflower (Spider-Man: Into the Spider-Verse),2018-10-18,day,1,album,spotify:album:47LpgGVshd0tbFSbm9tTLb,USUM71814888,https://open.spotify.com/track/1A6OTy97kk0mMdm...


In [23]:
track_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9105 entries, 0 to 9104
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   artists                       9105 non-null   object
 1   available_markets             9105 non-null   object
 2   disc_number                   9105 non-null   int64 
 3   duration_ms                   9105 non-null   int64 
 4   explicit                      9105 non-null   bool  
 5   href                          9105 non-null   object
 6   id                            9105 non-null   object
 7   is_local                      9105 non-null   bool  
 8   name                          9105 non-null   object
 9   popularity                    9105 non-null   int64 
 10  preview_url                   0 non-null      object
 11  track_number                  9105 non-null   int64 
 12  type                          9105 non-null   object
 13  uri               

In [43]:
track_data.head()

,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,preview_url,track_number,type,uri,album.album_type,album.artists,album.available_markets,album.external_urls.spotify,album.href,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,200040,False,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,0VjIjW4GlUZAMYd2vXMi3b,False,Blinding Lights,87,None,9,track,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4yP0hdKOZPNshxU...,https://api.spotify.com/v1/albums/4yP0hdKOZPNs...,4yP0hdKOZPNshxUOjY0cZj,[{'url': 'https://i.scdn.co/image/ab67616d0000...,After Hours,2020-03-20,day,14,album,spotify:album:4yP0hdKOZPNshxUOjY0cZj,USUG11904206,https://open.spotify.com/track/0VjIjW4GlUZAMYd...
1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,167303,False,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,4Dvkj6JhhA12EX05fT7y2e,False,As It Was,86,None,4,track,spotify:track:4Dvkj6JhhA12EX05fT7y2e,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5r36AJ6VOJtp00o...,https://api.spotify.com/v1/albums/5r36AJ6VOJtp...,5r36AJ6VOJtp00oxSkBZ5h,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Harry's House,2022-05-20,day,13,album,spotify:album:5r36AJ6VOJtp00oxSkBZ5h,USSM12200612,https://open.spotify.com/track/4Dvkj6JhhA12EX0...
2,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,182160,False,https://api.spotify.com/v1/tracks/7qEHsqek33rT...,7qEHsqek33rTcFNT9PFqLf,False,Someone You Loved,85,None,4,track,spotify:track:7qEHsqek33rTcFNT9PFqLf,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/5658aM19fA3JVwT...,https://api.spotify.com/v1/albums/5658aM19fA3J...,5658aM19fA3JVwTK6eQX70,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Divinely Uninspired To A Hellish Extent,2019-05-17,day,12,album,spotify:album:5658aM19fA3JVwTK6eQX70,DEUM71807062,https://open.spotify.com/track/7qEHsqek33rTcFN...
3,[],1,141805,True,https://api.spotify.com/v1/tracks/5PjdY0CKGZdE...,5PjdY0CKGZdEuoNab3yDmX,False,STAY (with Justin Bieber),10,None,3,track,spotify:track:5PjdY0CKGZdEuoNab3yDmX,album,[{'external_urls': {'spotify': 'https://open.s...,[],https://open.spotify.com/album/4bZJWQhHKJckFLJ...,https://api.spotify.com/v1/albums/4bZJWQhHKJck...,4bZJWQhHKJckFLJuYdvyX2,[{'url': 'https://i.scdn.co/image/ab67616d0000...,F*CK LOVE 3: OVER YOU,2021-07-23,day,29,album,spotify:album:4bZJWQhHKJckFLJuYdvyX2,USSM12103949,https://open.spotify.com/track/5PjdY0CKGZdEuoN...
4,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,157560,False,https://api.spotify.com/v1/tracks/0RiRZpuVRbi7...,0RiRZpuVRbi7oqRdSMwhQY,False,Sunflower - Spider-Man: Into the Spider-Verse,81,None,12,track,spotify:track:0RiRZpuVRbi7oqRdSMwhQY,album,[{'external_urls': {'spotify': 'https://open.s...,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",https://open.spotify.com/album/4g1ZRSobMefqF6n...,https://api.spotify.com/v1/albums/4g1ZRSobMefq...,4g1ZRSobMefqF6nelkgibi,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Hollywood's Bleeding,2019-09-06,day,17,album,spotify:album:4g1ZRSobMefqF6nelkgibi,USUM71814888,https://open.spotify.com/track/0RiRZpuVRbi7oqR...


In [24]:
selected_columns = [
    'duration_ms',
    'explicit',
    'is_local',
    'popularity',
    'track_number',
    'album.release_date',
    'album.total_tracks',
    'album.name', 
    'uri'
]
filtered_df = track_data[selected_columns].copy()
filtered_df.rename(columns={
    'album.release_date': 'release_date',
    'album.total_tracks': 'album_total_tracks',
    'album.name': 'album_name',
    'uri': 'spotify_uri' 
}, inplace=True)


In [25]:
filtered_df.head()

,duration_ms,explicit,is_local,popularity,track_number,release_date,album_total_tracks,album_name,spotify_uri
0,201573,False,False,22,1,2019-11-29,1,Blinding Lights,spotify:track:0sf12qNH5qcw8qpgymFOqD
1,167303,False,False,16,1,2022-03-31,1,As It Was,spotify:track:4LRPiXqCikLlN15c3yImP7
2,182160,False,False,70,3,2018-11-08,4,Breach,spotify:track:2TIlqbIneP0ZY1O0EzYLlc
3,233712,False,False,87,4,2017-03-03,16,÷ (Deluxe),spotify:track:7qiZfU4dY1lWllzX7mPBI3
4,158053,False,False,3,1,2018-10-18,1,Sunflower (Spider-Man: Into the Spider-Verse),spotify:track:1A6OTy97kk0mMdm78rHsm8


In [26]:
df.head()

,artist_names,track_name,peak_streams,total_streams,max_peak_rank,min_peak_rank,max_days_on_chart,best_day_streams,first_appearance,uri,source,count,best_day_date
0,The Weeknd,Blinding Lights,8453567,4552239347,8,1,1858,8453567,2019-11-29,spotify:track:0sf12qNH5qcw8qpgymFOqD,Republic Records,1858,2020-03-20
1,Harry Styles,As It Was,16103849,3640335748,40,1,1002,16103849,2022-03-31,spotify:track:4LRPiXqCikLlN15c3yImP7,Columbia,1002,2022-04-01
2,Lewis Capaldi,Someone You Loved,3974554,3625415177,198,4,2166,3974554,2019-01-07,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,Vertigo Berlin,2166,2019-10-30
3,Ed Sheeran,Shape of You,9891056,3607629552,1,1,2407,9891056,2017-01-06,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Atlantic Records UK,2407,2017-03-03
4,"Post Malone, Swae Lee",Sunflower - Spider-Man: Into the Spider-Verse,5033261,3467020076,196,1,2187,5033261,2018-10-18,spotify:track:1A6OTy97kk0mMdm78rHsm8,Republic Records,2187,2019-01-18


In [27]:

merged_df = pd.concat([df, filtered_df], axis=1)

merged_df.head()


,artist_names,track_name,peak_streams,total_streams,max_peak_rank,min_peak_rank,max_days_on_chart,best_day_streams,first_appearance,uri,source,count,best_day_date,duration_ms,explicit,is_local,popularity,track_number,release_date,album_total_tracks,album_name,spotify_uri
0,The Weeknd,Blinding Lights,8453567,4552239347,8,1,1858,8453567,2019-11-29,spotify:track:0sf12qNH5qcw8qpgymFOqD,Republic Records,1858,2020-03-20,201573,False,False,22,1,2019-11-29,1,Blinding Lights,spotify:track:0sf12qNH5qcw8qpgymFOqD
1,Harry Styles,As It Was,16103849,3640335748,40,1,1002,16103849,2022-03-31,spotify:track:4LRPiXqCikLlN15c3yImP7,Columbia,1002,2022-04-01,167303,False,False,16,1,2022-03-31,1,As It Was,spotify:track:4LRPiXqCikLlN15c3yImP7
2,Lewis Capaldi,Someone You Loved,3974554,3625415177,198,4,2166,3974554,2019-01-07,spotify:track:2TIlqbIneP0ZY1O0EzYLlc,Vertigo Berlin,2166,2019-10-30,182160,False,False,70,3,2018-11-08,4,Breach,spotify:track:2TIlqbIneP0ZY1O0EzYLlc
3,Ed Sheeran,Shape of You,9891056,3607629552,1,1,2407,9891056,2017-01-06,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Atlantic Records UK,2407,2017-03-03,233712,False,False,87,4,2017-03-03,16,÷ (Deluxe),spotify:track:7qiZfU4dY1lWllzX7mPBI3
4,"Post Malone, Swae Lee",Sunflower - Spider-Man: Into the Spider-Verse,5033261,3467020076,196,1,2187,5033261,2018-10-18,spotify:track:1A6OTy97kk0mMdm78rHsm8,Republic Records,2187,2019-01-18,158053,False,False,3,1,2018-10-18,1,Sunflower (Spider-Man: Into the Spider-Verse),spotify:track:1A6OTy97kk0mMdm78rHsm8


In [28]:
matching_rows = merged_df[merged_df['uri'] != merged_df['spotify_uri']]

print(matching_rows)

Empty DataFrame
Columns: [artist_names, track_name, peak_streams, total_streams, max_peak_rank, min_peak_rank, max_days_on_chart, best_day_streams, first_appearance, uri, source, count, best_day_date, duration_ms, explicit, is_local, popularity, track_number, release_date, album_total_tracks, album_name, spotify_uri]
Index: []


In [29]:

merged_df.to_csv('cleaned_global19-24_Spotify.csv', index=False)
